In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_excel("./Datasets/HVAC_limpio.xlsx", "HVAC_limpio", index_col=0)
#data

In [3]:
data['Fecha- hora de lectura'] = (data['Fecha- hora de lectura'] - data['Fecha- hora de lectura'].min())  / np.timedelta64(1,'D')
#data.drop(['Fecha- hora de lectura'], axis=1, inplace=True)

In [4]:
data.columns

Index(['Fecha- hora de lectura', 'POTENCIA TRAFO 2', 'POTENCIA TRAFO 3',
       'POTENCIA TRAFO 4', 'POTENCIA TRAFO 5', 'POTENCIA MEDIA CONECTADA',
       'CONTROL FRÍO', 'POTENCIA BOMBA CALOR FELIPE',
       'POTENCIA BOMBA CALOR CARLOS',
       'TEMPERATURA AMBIENTE BOMBA CALOR CARLOS',
       'TEMPERATURA AMBIENTE BOMBA CALOR FELIPE', 'TEMPERATURA EXTERIOR',
       'CAPACIDAD GRUPO DE FRÍO 2', 'POTENCIA GRUPO FRÍO 1',
       'POTENCIA GRUPO FRÍO 2', 'ENTRADA AGUA A TORRE 1',
       'SALIDA AGUA TORRE 1', 'ENTRADA AGUA A TORRE 2', 'SALIDA AGUA TORRE 2',
       'C_O_P MÁQUINA GRUPO FRÍO 1', 'C_O_P MÁQUINA GRUPO FRÍO 2',
       'C_O_P BOMBA CALOR CARLOS', 'C_O_P BOMBA CALOR FELIPE',
       'TEMPERATURA SALIDA BOMBA CALOR CARLOS',
       'TEMPERATURA SALIDA BOMBA CALOR FELIPE',
       'KILO CALORÍAS GENERADAS BOMBA CALOR CARLOS',
       'KILO CALORÍAS GENERADAS BOMBA CALOR FELIPE',
       'KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 1',
       'KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 2',
    

In [5]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim != None:
        plt.ylim(*ylim)
    plt.xlabel("Training samples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Train score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Test score")

    plt.legend(loc="best")
    return plt

## MLP Regressor

In [157]:
from sklearn.neural_network import MLPRegressor

def mLPRegressor(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    mlp = MLPRegressor(activation='logistic', solver='lbfgs')
    mlp.fit(X_train, y_train)
    predicts = mlp.predict(X_test)
    acc = mlp.score(X_test, y_test)
    mse = mean_squared_error(y_test, predicts)
    print("Accuracy: ", acc)
    print("MSE: ", mse)
    print("---------------------------------------")
    #print(predicts.tolist())
    #print(y_test.tolist())
    return mlp

In [158]:
X = data[['TEMPERATURA SALIDA BOMBA CALOR CARLOS','POTENCIA TERMICA BOMBA CALOR CARLOS', 
          'C_O_P BOMBA CALOR CARLOS', 'POTENCIA TRAFO 4','POTENCIA TRAFO 5',
          'TEMPERATURA EXTERIOR', 'TEMPERATURA AMBIENTE BOMBA CALOR CARLOS']]
y = data['POTENCIA BOMBA CALOR CARLOS']
mlpCalorCarlos = mLPRegressor(X, y)

Accuracy:  0.9144232199295135
MSE:  163.97858569292748
---------------------------------------


In [159]:
X = data[['TEMPERATURA SALIDA BOMBA CALOR FELIPE','POTENCIA TERMICA BOMBA CALOR FELIPE', 
          'C_O_P BOMBA CALOR FELIPE', 'POTENCIA TRAFO 4','POTENCIA TRAFO 5',
          'TEMPERATURA EXTERIOR', 'TEMPERATURA AMBIENTE BOMBA CALOR FELIPE' ]]
y = data['POTENCIA BOMBA CALOR FELIPE']
mlpCalorFelipe = mLPRegressor(X, y)

Accuracy:  0.8806040162665498
MSE:  243.64851408530618
---------------------------------------


In [160]:
X = data[['POTENCIA TERMICA GRUPO FRIO 1', 'ENTRADA AGUA A TORRE 1', 'SALIDA AGUA TORRE 1',
          'POTENCIA TRAFO 4', 'POTENCIA TRAFO 5']]
y = data['POTENCIA GRUPO FRÍO 1']
mlpFrio1 = mLPRegressor(X, y)

Accuracy:  0.6986588426246625
MSE:  215.23498926362365
---------------------------------------


In [161]:
X = data[['POTENCIA TERMICA GRUPO FRIO 2', 'ENTRADA AGUA A TORRE 1', 'SALIDA AGUA TORRE 2',
          'POTENCIA TRAFO 4', 'POTENCIA TRAFO 5', 'POTENCIA MEDIA CONECTADA', 'CONTROL FRÍO',
          'KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 2', 'TEMPERATURA EXTERIOR']]
y = data['POTENCIA GRUPO FRÍO 2']
mlpGrio1 = mLPRegressor(X, y)

Accuracy:  0.004511904440945402
MSE:  809.3185936264903
---------------------------------------


## SGD Regressor

In [162]:
from sklearn.linear_model import SGDRegressor

def sGDRegressor(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    clf = SGDRegressor()
    clf.fit(X_train, y_train)
    predicts = clf.predict(X_test)
    acc = clf.score(X_test, y_test)
    mse = mean_squared_error(y_test, predicts)
    print("Accuracy: ", acc)
    print("MSE: ", mse)
    print("---------------------------------------")
    #print(predicts.tolist())
    return clf

In [163]:
X = data[['TEMPERATURA SALIDA BOMBA CALOR CARLOS','POTENCIA TERMICA BOMBA CALOR CARLOS', 
          'C_O_P BOMBA CALOR CARLOS', 'POTENCIA TRAFO 4','POTENCIA TRAFO 5',
          'TEMPERATURA EXTERIOR', 'TEMPERATURA AMBIENTE BOMBA CALOR CARLOS']]
y = data['POTENCIA BOMBA CALOR CARLOS']
clfCalorCarlos = sGDRegressor(X, y)

Accuracy:  -1.1116850616254075e+23
MSE:  2.1391588295824012e+26
---------------------------------------


In [164]:
X = data[['TEMPERATURA SALIDA BOMBA CALOR FELIPE','POTENCIA TERMICA BOMBA CALOR FELIPE', 
          'C_O_P BOMBA CALOR FELIPE', 'POTENCIA TRAFO 4','POTENCIA TRAFO 5',
          'TEMPERATURA EXTERIOR', 'TEMPERATURA AMBIENTE BOMBA CALOR FELIPE' ]]
y = data['POTENCIA BOMBA CALOR FELIPE']
clfCalorFelipe = sGDRegressor(X, y)

Accuracy:  -3.017991175877767e+22
MSE:  6.07116025751942e+25
---------------------------------------


In [165]:
X = data[['POTENCIA TERMICA GRUPO FRIO 1', 'ENTRADA AGUA A TORRE 1', 'SALIDA AGUA TORRE 1',
          'POTENCIA TRAFO 4', 'POTENCIA TRAFO 5']]
y = data['POTENCIA GRUPO FRÍO 1']
clfGrupo1 = sGDRegressor(X, y)

Accuracy:  -1.9906239379120934e+24
MSE:  1.4408052548262501e+27
---------------------------------------


In [166]:
X = data[['POTENCIA TERMICA GRUPO FRIO 2', 'ENTRADA AGUA A TORRE 1', 'SALIDA AGUA TORRE 2',
          'POTENCIA TRAFO 4', 'POTENCIA TRAFO 5', 'POTENCIA MEDIA CONECTADA', 'CONTROL FRÍO',
          'KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 2', 'TEMPERATURA EXTERIOR']]
y = data['POTENCIA GRUPO FRÍO 2']
clfGrupo2 = sGDRegressor(X, y)

Accuracy:  -2.1293677234193948e+33
MSE:  1.7285671478382134e+36
---------------------------------------


## KNeighbors Regressor

In [168]:
from sklearn.neighbors import KNeighborsRegressor

def kNRegressor(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    neigh = KNeighborsRegressor(n_neighbors=10)
    neigh.fit(X_train, y_train) 
    predicts = neigh.predict(X_test)
    acc = neigh.score(X_test, y_test)
    mse = mean_squared_error(y_test, predicts)
    print("Accuracy: ", acc)
    print("MSE: ", mse)
    print("---------------------------------------")
    #print(predicts.tolist())
    #print(y_test.tolist())
    return neigh

In [169]:
X = data[['TEMPERATURA SALIDA BOMBA CALOR CARLOS','POTENCIA TERMICA BOMBA CALOR CARLOS', 
          'C_O_P BOMBA CALOR CARLOS', 'POTENCIA TRAFO 4','POTENCIA TRAFO 5',
          'TEMPERATURA EXTERIOR', 'TEMPERATURA AMBIENTE BOMBA CALOR CARLOS']]
y = data['POTENCIA BOMBA CALOR CARLOS']
knrCalorCarlos = kNRegressor(X, y)

Accuracy:  0.9656927574582698
MSE:  65.81355759299234
---------------------------------------


In [170]:
X = data[['TEMPERATURA SALIDA BOMBA CALOR FELIPE','POTENCIA TERMICA BOMBA CALOR FELIPE', 
          'C_O_P BOMBA CALOR FELIPE', 'POTENCIA TRAFO 4','POTENCIA TRAFO 5',
          'TEMPERATURA EXTERIOR', 'TEMPERATURA AMBIENTE BOMBA CALOR FELIPE' ]]
y = data['POTENCIA BOMBA CALOR FELIPE']
knrCalorFelipe = kNRegressor(X, y)

Accuracy:  0.9646740022222274
MSE:  72.2207299928767
---------------------------------------


In [171]:
X = data[['POTENCIA TERMICA GRUPO FRIO 1', 'ENTRADA AGUA A TORRE 1', 'SALIDA AGUA TORRE 1',
          'POTENCIA TRAFO 4', 'POTENCIA TRAFO 5', 'POTENCIA MEDIA CONECTADA', 'CONTROL FRÍO',
          'KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 1', 'TEMPERATURA EXTERIOR']]
y = data['POTENCIA GRUPO FRÍO 1']
knrGrupo1 = kNRegressor(X, y)

Accuracy:  0.8776554676192232
MSE:  87.56708833429023
---------------------------------------


In [172]:
X = data[['POTENCIA TERMICA GRUPO FRIO 2', 'ENTRADA AGUA A TORRE 1', 'SALIDA AGUA TORRE 2',
          'POTENCIA TRAFO 4', 'POTENCIA TRAFO 5', 'POTENCIA MEDIA CONECTADA', 'CONTROL FRÍO',
          'KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 2', 'TEMPERATURA EXTERIOR']]
y = data['POTENCIA GRUPO FRÍO 2']
knrGrupo2 = kNRegressor(X, y)

Accuracy:  0.9497885561012611
MSE:  40.85149691643758
---------------------------------------


## Gradient Boosting Regressor

In [173]:
from sklearn.ensemble import GradientBoostingRegressor

def gradientBoostingRegressor(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    est = GradientBoostingRegressor()
    est.fit(X_train, y_train)
    predicts = est.predict(X_test)
    acc = est.score(X_test, y_test)
    mse = mean_squared_error(y_test, predicts)
    print("Accuracy: ", acc)
    print("MSE: ", mse)
    print("---------------------------------------")
    #print(predicts.tolist())
    #print(y_test.tolist())
    return est

In [174]:
X = data[['TEMPERATURA SALIDA BOMBA CALOR CARLOS','POTENCIA TERMICA BOMBA CALOR CARLOS', 
          'C_O_P BOMBA CALOR CARLOS', 'POTENCIA TRAFO 4','POTENCIA TRAFO 5',
          'TEMPERATURA EXTERIOR', 'TEMPERATURA AMBIENTE BOMBA CALOR CARLOS']]
y = data['POTENCIA BOMBA CALOR CARLOS']
estCalorCarlos = gradientBoostingRegressor(X, y)

Accuracy:  0.9786826734969578
MSE:  40.957025810896226
---------------------------------------


In [175]:
X = data[['TEMPERATURA SALIDA BOMBA CALOR FELIPE','POTENCIA TERMICA BOMBA CALOR FELIPE', 
          'C_O_P BOMBA CALOR FELIPE', 'POTENCIA TRAFO 4','POTENCIA TRAFO 5',
          'TEMPERATURA EXTERIOR', 'TEMPERATURA AMBIENTE BOMBA CALOR FELIPE' ]]
y = data['POTENCIA BOMBA CALOR FELIPE']
estCalorFelipe = gradientBoostingRegressor(X, y)

Accuracy:  0.9639569380193129
MSE:  72.52505221190981
---------------------------------------


In [176]:
X = data[['POTENCIA TERMICA GRUPO FRIO 1', 'ENTRADA AGUA A TORRE 1', 'SALIDA AGUA TORRE 1',
          'POTENCIA TRAFO 4', 'POTENCIA TRAFO 5', 'POTENCIA MEDIA CONECTADA', 'CONTROL FRÍO',
          'KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 1', 'TEMPERATURA EXTERIOR']]
y = data['POTENCIA GRUPO FRÍO 1']
estGrupo1 = gradientBoostingRegressor(X, y)

Accuracy:  0.9181038459757389
MSE:  58.98735452345555
---------------------------------------


In [177]:
X = data[['POTENCIA TERMICA GRUPO FRIO 2', 'ENTRADA AGUA A TORRE 1', 'SALIDA AGUA TORRE 2',
          'POTENCIA TRAFO 4', 'POTENCIA TRAFO 5', 'POTENCIA MEDIA CONECTADA', 'CONTROL FRÍO',
          'KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 2', 'TEMPERATURA EXTERIOR']]
y = data['POTENCIA GRUPO FRÍO 2']
estGrupo2 = gradientBoostingRegressor(X, y)

Accuracy:  0.9493021631258106
MSE:  41.66168224737247
---------------------------------------


## Ranfom Forest Regressor

In [178]:
from sklearn.ensemble import RandomForestRegressor

def randomForestRegressor(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    regr = RandomForestRegressor(n_estimators=10)
    regr.fit(X_train, y_train)
    predicts = regr.predict(X_test)
    acc = regr.score(X_test, y_test)
    mse = mean_squared_error(y_test, predicts)
    r2 = r2_score(y_test, predicts) 
    print("Accuracy: ", acc)
    print("MSE: ", mse)
    print("R2: ", r2)
    print("---------------------------------------")
    #print(predicts.tolist())
    #print(y_test.tolist())
    return regr

In [179]:
X = data[['TEMPERATURA SALIDA BOMBA CALOR CARLOS','POTENCIA TERMICA BOMBA CALOR CARLOS', 
          'C_O_P BOMBA CALOR CARLOS', 'POTENCIA TRAFO 4','POTENCIA TRAFO 5',
          'TEMPERATURA EXTERIOR', 'TEMPERATURA AMBIENTE BOMBA CALOR CARLOS' ]]
y = data['POTENCIA BOMBA CALOR CARLOS']
regrCalorCarlos = randomForestRegressor(X, y)

Accuracy:  0.9917830325606767
MSE:  15.849832611849141
R2:  0.9917830325606766
---------------------------------------


In [180]:
X = data[['TEMPERATURA SALIDA BOMBA CALOR FELIPE','POTENCIA TERMICA BOMBA CALOR FELIPE', 
          'C_O_P BOMBA CALOR FELIPE', 'POTENCIA TRAFO 4','POTENCIA TRAFO 5',
          'TEMPERATURA EXTERIOR', 'TEMPERATURA AMBIENTE BOMBA CALOR FELIPE' ]]
y = data['POTENCIA BOMBA CALOR FELIPE']
regrCalorFelipe = randomForestRegressor(X, y)

Accuracy:  0.9864096117775011
MSE:  28.062117775692315
R2:  0.9864096117775011
---------------------------------------


In [181]:
X = data[['POTENCIA TERMICA GRUPO FRIO 1', 'ENTRADA AGUA A TORRE 1', 'SALIDA AGUA TORRE 1',
          'POTENCIA TRAFO 4', 'POTENCIA TRAFO 5', 'POTENCIA MEDIA CONECTADA', 'CONTROL FRÍO',
          'KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 1', 'TEMPERATURA EXTERIOR']]
y = data['POTENCIA GRUPO FRÍO 1']
regrGrupo1 = randomForestRegressor(X, y)

Accuracy:  0.9752901957487665
MSE:  17.724885325218203
R2:  0.9752901957487665
---------------------------------------


In [182]:
X = data[['POTENCIA TERMICA GRUPO FRIO 2', 'ENTRADA AGUA A TORRE 1', 'SALIDA AGUA TORRE 2',
          'POTENCIA TRAFO 4', 'POTENCIA TRAFO 5', 'POTENCIA MEDIA CONECTADA', 'CONTROL FRÍO',
          'KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 2', 'TEMPERATURA EXTERIOR']]
y = data['POTENCIA GRUPO FRÍO 2']
regrGrupo2 = randomForestRegressor(X, y)

Accuracy:  0.9873926118393384
MSE:  10.242829387066246
R2:  0.9873926118393385
---------------------------------------
